# Collaborative filtering on Movielens

## Item-to-item collaborative filtering

$$
         w(i, j) = \frac{\sum_{u\in U_i \cap U_j}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U_i \cap U_j} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U_i \cap U_j} (r_{u,j}-\bar{r}_u)^2}}.
$$

\begin{equation}
 w(c,I_u) = \sum_{i\in I_u} w_{c,i}.
\end{equation}


### Import useful requirements

In [1]:
import os

if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    !wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip
    !unzip recsys.zip

--2023-10-25 09:39:37--  https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nzhinusoftcm/review-on-collaborative-filtering/master/recsys.zip [following]
--2023-10-25 09:39:38--  https://raw.githubusercontent.com/nzhinusoftcm/review-on-collaborative-filtering/master/recsys.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15312323 (15M) [application/zip]
Saving to: ‘recsys.zip’

recsys.zip          100%[===================>]  14.60M  8.58MB/s    in 1.7s    

2023-10-25 09:39:41 (8.58 MB/s) - ‘recsys.zip’ sa

### Import requirements

In [1]:
import os
import sys
import typing as tp

import joblib
import numpy as np
import pandas as pd
import tqdm.notebook
from recsys.datasets import ml1m, ml100k
from sklearn.preprocessing import LabelEncoder

### Dataset upload\

In [2]:
%load_ext autoreload
%autoreload 2

Will use movielens dataset

In [3]:
ratings, movies = ml100k.load()

Download data 100.2%
Successfully downloaded ml-100k.zip 4924029 bytes.
Unzipping the ml-100k.zip zip file ...


In [4]:
ratings.head()

,userid,itemid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [5]:
movies.head()

,itemid,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


### Preprocessing

In [6]:
def ids_encoder(ratings):
    users = sorted(ratings["userid"].unique())
    items = sorted(ratings["itemid"].unique())

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    # encode userids and itemids
    ratings.userid = uencoder.transform(ratings.userid.tolist())
    ratings.itemid = iencoder.transform(ratings.itemid.tolist())

    return ratings, uencoder, iencoder

In [7]:
# create the encoder
ratings, uencoder, iencoder = ids_encoder(ratings)

## Implementation

### Part 1. Similarities

\begin{equation}
 w_{i,j}= \frac{\sum_{u\in U}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U} (r_{u,j}-\bar{r}_u)^2}}.
\end{equation}

In [8]:
def normalize(ratings: pd.DataFrame) -> pd.DataFrame:
    """Normalize ratings by user"""
    # calculate mean for every user
    mean = ratings.groupby(by="userid", as_index=False)["rating"].mean()
    norm_ratings = pd.merge(ratings, mean, suffixes=("", "_mean"), on="userid")

    # normalize each rating by substracting the mean rating of the corresponding user
    norm_ratings["norm_rating"] = norm_ratings["rating"] - norm_ratings["rating_mean"]

    return norm_ratings[ratings.columns.tolist() + ["norm_rating"]]

In [9]:
def test_normalize():
    test_df = pd.DataFrame(
        {
            "userid": [0, 0, 0, 1, 1],
            "itemid": [0, 1, 2, 1, 3],
            "rating": [2, 2, 5, 5, 5],
        }
    )

    expected = pd.DataFrame(
        {
            "userid": [0, 0, 0, 1, 1],
            "itemid": [0, 1, 2, 1, 3],
            "rating": [2, 2, 5, 5, 5],
            "norm_rating": [-1, -1, 2, 0, 0],
        }
    )

    assert (
        test_df.shape[0] == expected.shape[0]
    ), "Number of user-item interactions is different"
    assert test_df.shape[1] + 1 == expected.shape[1], "Number of columns is incorrect"
    assert (normalize(test_df) == expected).all().all(), "Result is incorrect"


test_normalize()

In [10]:
norm_ratings = normalize(ratings)
np_ratings = norm_ratings.to_numpy()
norm_ratings.head()

,userid,itemid,rating,norm_rating
0,0,0,5,1.389706
1,0,1,3,-0.610294
2,0,2,4,0.389706
3,0,3,3,-0.610294
4,0,4,3,-0.610294


In [11]:
def cosine(x: np.array, y: np.array) -> float:
    if np.linalg.norm(x) == 0 or np.linalg.norm(y) == 0:
        return 0

    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [14]:
from functools import lru_cache


@lru_cache(2000)
def ratings_for_item(i):
    return np_ratings[np_ratings[:, 1] == i]


def calculate_similarity_between_two(np_ratings: np.array, i: int, j: int) -> float:
    """
    np_ratings: array containing: (user_id, item_id, rating, rating_mean, norm_rating)
    i: index of the first item
    j: index of the second item

    Returns:
        pearson correlation between i and j
    """
    if i == j:
        return 1.0

    ratings_i, ratings_j = ratings_for_item(i), ratings_for_item(j)

    common_users = np.intersect1d(ratings_i[:, 0], ratings_j[:, 0])

    if len(common_users) == 0:
        return -1.0  # 0.0

    common_ratings_i = ratings_i[np.isin(ratings_i[:, 0], common_users)]
    common_ratings_j = ratings_j[np.isin(ratings_j[:, 0], common_users)]

    assert sorted(common_ratings_i[:, 0]) == sorted(common_ratings_j[:, 0])

    x = common_ratings_i[:, 3]
    y = common_ratings_j[:, 3]
    return cosine(x, y)

In [45]:
assert np.isclose(calculate_similarity_between_two(np_ratings, 0, 0), 1.0)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 2), 0.1069226)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 3), 0.0555092)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 5), -0.125509)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 1431), 1.0)
assert np.isclose(calculate_similarity_between_two(np_ratings, 4, 1123), 0.0)

In [46]:
def adjusted_cosine(
    np_ratings: np.array, similarity_between_two
) -> tp.Tuple[np.array, np.array]:
    """Computes correlation for all pairs

    np_ratings: array containing: (user_id, item_id, rating, rating_mean, norm_rating)
    similarity_between_two: function to calculate similarity
    """
    nb_items = np.unique(np_ratings[:, 1]).size
    similarities = np.full((nb_items, nb_items), -1)
    np.fill_diagonal(similarities, 1)

    items = sorted(set(map(int, np_ratings[:, 1])))

    with tqdm.notebook.tqdm(total=len(items) * (len(items) - 1) // 2) as pbar:
        for i in range(len(items)):
            for j in range(i + 1, len(items)):
                sim = similarity_between_two(np_ratings, items[i], items[j])
                similarities[items[i], items[j]] = sim
                similarities[items[j], items[i]] = sim
                pbar.update()

    assert np.all(
        similarities.T == similarities
    ), "Similarity matrix should be symmetrical"
    assert np.allclose(
        np.diag(similarities), 1.0
    ), "Similarities of items with themselves should be 1"

    # get neighbors by their neighbors in decreasing order of similarities
    neighbors = np.flip(np.argsort(similarities), axis=1)

    # sort similarities in decreasing order
    similarities = np.flip(np.sort(similarities), axis=1)

    return similarities, neighbors

In [47]:
similarities, neighbors = adjusted_cosine(np_ratings, calculate_similarity_between_two)

  0%|          | 0/1413721 [00:00<?, ?it/s]

In [48]:
np.unique(similarities[100])

array([-1,  0,  1])

In [34]:
def neighbours_viz(
    item_id: int,
    movies: pd.DataFrame,
    similarities: np.array,
    neighbours: np.array,
    k=5,
):
    orig_index = iencoder.inverse_transform([item_id])[0]
    film_name = movies[movies.itemid == orig_index].title.values[0]
    similar_films = (
        (
            neighbor_id,
            movies[
                movies.itemid == iencoder.inverse_transform([neighbor_id])[0]
            ].title.values[0],
            similarity,
        )
        for neighbor_id, similarity in zip(
            neighbors[item_id][:k], similarities[item_id][:k]
        )
    )
    display(
        pd.DataFrame(
            dict(zip(("item_id", film_name, "Similarity"), zip(*similar_films)))
        )
    )
    print("\n")

In [39]:
neighbours_viz(49, movies, similarities, neighbors)
neighbours_viz(68, movies, similarities, neighbors)
neighbours_viz(914, movies, similarities, neighbors)
neighbours_viz(319, movies, similarities, neighbors)
neighbours_viz(200, movies, similarities, neighbors)

,item_id,Star Wars (1977),Similarity
0,1632,Á köldum klaka (Cold Fever) (1994),1
1,599,Daniel Defoe's Robinson Crusoe (1996),1
2,1457,"Damsel in Distress, A (1937)",1
3,1620,Butterfly Kiss (1995),1
4,1129,Jupiter's Wife (1994),1


,item_id,Forrest Gump (1994),Similarity
0,1525,Witness (1985),1
1,991,Head Above Water (1996),1
2,1256,"Designated Mourner, The (1997)",1
3,1255,"Designated Mourner, The (1997)",1
4,1462,"Boys, Les (1997)",1


,item_id,Primary Colors (1998),Similarity
0,1438,Jason's Lyric (1994),1
1,1405,When Night Is Falling (1995),1
2,262,Steel (1997),1
3,1394,Hurricane Streets (1998),1
4,578,Fatal Instinct (1993),1


,item_id,Paradise Lost: The Child Murders at Robin Hood Hills (1996),Similarity
0,1000,"Stupids, The (1996)",1
1,1395,Stonewall (1995),1
2,1385,Tetsuo II: Body Hammer (1992),1
3,1387,Gabbeh (1996),1
4,1034,Cool Runnings (1993),1


,item_id,Evil Dead II (1987),Similarity
0,1259,Total Eclipse (1995),1
1,896,Time Tracers (1995),1
2,882,Telling Lies in America (1997),1
3,883,Year of the Horse (1997),1
4,1067,"Star Maker, The (Uomo delle stelle, L') (1995)",1


In [50]:
def calculate_similarity_between_two_with_threshold(
    np_ratings: np.array, i: int, j: int
) -> float:
    if i == j:
        return 1.0

    ratings_i, ratings_j = ratings_for_item(i), ratings_for_item(j)
    common_users = np.intersect1d(ratings_i[:, 0], ratings_j[:, 0])
    common_ratings_i = ratings_i[np.isin(ratings_i[:, 0], common_users)]
    common_ratings_j = ratings_j[np.isin(ratings_j[:, 0], common_users)]

    if len(common_users) > 20:
        assert sorted(common_ratings_i[:, 0]) == sorted(common_ratings_j[:, 0])
        x = common_ratings_i[:, 3]
        y = common_ratings_j[:, 3]
        return cosine(x, y)
    return -1.0

In [51]:
assert np.isclose(
    calculate_similarity_between_two_with_threshold(np_ratings, 1, 1431), -1.0
)
assert np.isclose(
    calculate_similarity_between_two_with_threshold(np_ratings, 1, 17), -1.0
)
assert np.isclose(
    calculate_similarity_between_two_with_threshold(np_ratings, 4, 1123), -1.0
)
assert np.isclose(
    calculate_similarity_between_two_with_threshold(np_ratings, 914, 1681), -1.0
)

In [52]:
similarities, neighbors = adjusted_cosine(
    np_ratings, calculate_similarity_between_two_with_threshold
)

  0%|          | 0/1413721 [00:00<?, ?it/s]

In [ ]:
neighbours_viz(49, movies, similarities, neighbors)
neighbours_viz(68, movies, similarities, neighbors)
neighbours_viz(154, movies, similarities, neighbors)
neighbours_viz(200, movies, similarities, neighbors)

,item_id,Star Wars (1977),Similarity
0,49,Star Wars (1977),1.000000
1,171,"Empire Strikes Back, The (1980)",0.826287
2,180,Return of the Jedi (1983),0.728182
3,173,Raiders of the Lost Ark (1981),0.714250
4,407,"Close Shave, A (1995)",0.659379


,item_id,Forrest Gump (1994),Similarity
0,68,Forrest Gump (1994),1.000000
1,214,Field of Dreams (1989),0.445981
2,309,"Rainmaker, The (1997)",0.436435
3,21,Braveheart (1995),0.422572
4,965,"Affair to Remember, An (1957)",0.420235


,item_id,Dirty Dancing (1987),Similarity
0,154,Dirty Dancing (1987),1.000000
1,626,Robin Hood: Prince of Thieves (1991),0.727235
2,568,Wolf (1994),0.716361
3,254,My Best Friend's Wedding (1997),0.708146
4,416,"Parent Trap, The (1961)",0.634029


,item_id,Evil Dead II (1987),Similarity
0,200,Evil Dead II (1987),1.000000
1,183,Army of Darkness (1993),0.574317
2,23,Rumble in the Bronx (1995),0.536677
3,90,"Nightmare Before Christmas, The (1993)",0.491057
4,557,Heavenly Creatures (1994),0.481768


### Part 2. Top items for a user

In [54]:
def candidate_items(
    np_ratings: np.array, userid: int, k=-1
) -> tp.Tuple[np.array, np.array]:
    # 1. Finding the set I_u of items already rated by user userid
    I_u = np_ratings[np_ratings[:, 0] == userid]
    I_u = I_u[:, 1].astype("int")

    # 2. Taking the union of similar items for all items in I_u to form the set of candidate items
    c = set()

    for iid in I_u:
        # add the neighbors of item iid in the set of candidate items
        c.update(neighbors[iid, :k])

    c = list(c)
    # 3. exclude from the set C all items in I_u.
    candidates = np.setdiff1d(c, I_u, assume_unique=True)

    return I_u, candidates

In [58]:
i_u, u_candidates = candidate_items(np_ratings, uencoder.transform([3])[0])

print("Films seen by user:", len(i_u))
print("Candidates:", len(u_candidates))

Films seen by user: 54
Candidates: 1628


In [60]:
i_u, u_candidates = candidate_items(np_ratings, uencoder.transform([3])[0], 10)

print("Films seen by user:", len(i_u))
print("Candidates:", len(u_candidates))

Films seen by user: 54
Candidates: 150


In [61]:
def similarity_with_Iu(item_id: int, I_u: np.array) -> float:
    w = 0
    for iid in I_u:
        # get similarity between itemid and c, if c is one of the k nearest neighbors of itemid
        if item_id in neighbors[iid]:
            w = w + similarities[iid, neighbors[iid] == item_id][0]
    return w

In [63]:
def rank_candidates(candidates: np.array, I_u: np.array) -> np.array:
    # list of candidate items mapped to their corresponding similarities to I_u
    sims = [similarity_with_Iu(c, I_u) for c in candidates]
    candidates = iencoder.inverse_transform(candidates)
    mapping = list(zip(candidates, sims))

    ranked_candidates = sorted(mapping, key=lambda couple: couple[1], reverse=True)
    return ranked_candidates

## Putting it alltogether

In [65]:
def topn_recommendation(np_ratings: np.array, userid, k=-1, N=30):
    # find candidate items
    I_u, candidates = candidate_items(np_ratings, userid, k)

    # rank candidate items according to their similarities with I_u
    ranked_candidates = rank_candidates(candidates, I_u)

    # get the first N row of ranked_candidates to build the top N recommendation list
    topn = pd.DataFrame(ranked_candidates[:N], columns=["itemid", "similarity_with_Iu"])
    topn = pd.merge(topn, movies, on="itemid", how="inner")
    return topn

In [ ]:
topn_recommendation(np_ratings, uencoder.transform([1])[0])

,itemid,similarity_with_Iu,title
0,313,-16.941690,Titanic (1997)
1,318,-23.278784,Schindler's List (1993)
2,655,-24.349682,Stand by Me (1986)
3,357,-26.289772,One Flew Over the Cuckoo's Nest (1975)
4,433,-27.140886,Heathers (1989)
5,423,-27.654851,E.T. the Extra-Terrestrial (1982)
6,651,-27.970798,Glory (1989)
7,288,-29.451573,Scream (1996)
8,276,-29.451749,Leaving Las Vegas (1995)
9,527,-29.573808,Gandhi (1982)


In [66]:
test_history = [49, 81, 180, 256, 131, 379]
movies.iloc[test_history]

,itemid,title
49,50,Star Wars (1977)
81,82,Jurassic Park (1993)
180,181,Return of the Jedi (1983)
256,257,Men in Black (1997)
131,132,"Wizard of Oz, The (1939)"
379,380,Star Trek: Generations (1994)


In [67]:
def candidate_items_by_user_history(I_u: tp.List[int], k=-1):
    c = set()
    for iid in I_u:
        c.update(neighbors[iid, :k])
    candidates = np.setdiff1d(list(c), I_u, assume_unique=True)

    return candidates


def topn_recommendations_by_user_history(I_u: tp.List[int], k=-1, N=30):
    candidates = candidate_items_by_user_history(I_u, k=k)
    ranked_candidates = rank_candidates(candidates, I_u)
    topn = pd.DataFrame(ranked_candidates[:N], columns=["itemid", "similarity_with_Iu"])
    topn = pd.merge(topn, movies, on="itemid", how="inner")
    return topn


topn_recommendations_by_user_history(test_history)

,itemid,similarity_with_Iu,title
0,1,0,Toy Story (1995)
1,2,0,GoldenEye (1995)
2,3,0,Four Rooms (1995)
3,4,0,Get Shorty (1995)
4,5,0,Copycat (1995)
5,7,0,Twelve Monkeys (1995)
6,8,0,Babe (1995)
7,9,0,Dead Man Walking (1995)
8,10,0,Richard III (1995)
9,11,0,Seven (Se7en) (1995)
